# Welcome to Jupyter!

In [1]:
# !pip install nltk
# !pip install tensorflow
# !pip install keras
import sklearn as sk
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
from nltk.corpus import stopwords
import string
import nltk
import tensorflow
from sklearn.model_selection import train_test_split
from collections import Counter
from pandas import DataFrame
from matplotlib import pyplot
import re
from nltk.stem import WordNetLemmatizer 
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout


In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
def process_news(news):
    _news = news.replace('b\"', "")
    _news = _news.replace('b\'', "")
    _news = _news.lower()
    _news = re.sub("[^a-zA-Z]", " ",_news)
    _news = re.sub('[\s]+', ' ', _news)
    
    tokens = _news.split(" ")
    if "" in tokens:
        tokens.remove("")
    
    lemmatizer = WordNetLemmatizer() 
    tokens = [lemmatizer.lemmatize(w) for w in tokens]
    #remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    
    _news = ' '.join(tokens)    
     
    return _news

In [5]:
def read_data():

    data = pd.read_csv(r"Combined_News_DJIA.csv")
    
    dfs = []
    data["News"] = ""
    for i in range(1,25):
        col = "Top"+str(i)
        data["News"] = data["News"] +" "+ data[col]
    data = data.dropna()
    data['PreProcessedNews'] = data['News'].map(process_news)
    
    data = data[['Date', 'News', 'PreProcessedNews', 'Label']]
    
    stock_prices = "upload_DJIA_table.csv"
    stock_data = pd.read_csv(stock_prices)
    
    print(data.head(2))
    print(stock_data.head(2))
    
    
    #merged_dataframe = data.merge(stock_data, how='inner', on='Date')
    merged_dataframe = pd.merge(data, stock_data, how='inner', on = 'Date')

    
    
    
    Xy_train = merged_dataframe[:int(len(data)*0.8)]
    #Xy_valid = merged_dataframe[int(len(data)*0.6):int(len(data)*0.8)]
    Xy_test = merged_dataframe[int(len(data)*0.8):]
    
    return merged_dataframe, Xy_train, Xy_test

In [6]:
news, Xy_train, Xy_test = read_data()

         Date                                               News  \
0  2008-08-08   b"Georgia 'downs two Russian warplanes' as co...   
1  2008-08-11   b'Why wont America and Nato help us? If they ...   

                                    PreProcessedNews  Label  
0  georgia two russian warplane country move brin...      0  
1  wont america nato help wont help help iraq bus...      1  
         Date          Open          High           Low         Close  \
0  2016-07-01  17924.240234  18002.380859  17916.910156  17949.369141   
1  2016-06-30  17712.759766  17930.609375  17711.800781  17929.990234   

      Volume     Adj Close  
0   82160000  17949.369141  
1  133030000  17929.990234  


In [7]:
X_train = Xy_train['PreProcessedNews'].to_frame().reset_index()
#X_valid = Xy_valid['PreProcessedNews'].to_frame()
#X_train =pd.concat(X_train, X_valid)
X_test = Xy_test['PreProcessedNews'].to_frame().reset_index()
#Xy_train =pd.concat(Xy_train, Xy_valid) 
y_train = Xy_train['Label'].to_numpy()
#y_valid = Xy_valid['Label'].to_numpy()
y_test = Xy_test['Label'].to_numpy()

In [8]:
def evaluate_mode(Xtrain, ytrain, Xtest, ytest):
    scores = list()
    n_repeats = 5
    n_words = Xtest.shape[1]
    for i in range(n_repeats):
        # define network
        
        #This is where you modify the network parameters
        model = Sequential()
        model.add(Dense(75, input_shape=(n_words,), activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
        # compile network
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        # fit network
        model.fit(Xtrain, ytrain,validation_split = 0.25, epochs=10, verbose=2)
        # evaluate
        loss, acc = model.evaluate(Xtest, ytest, verbose=0)
        scores.append(acc)
        print('%d accuracy: %s' % ((i + 1), acc))
    return scores

In [9]:
def prepare_data(train_docs, test_docs, mode):
    # create the tokenizer
    tokenizer = Tokenizer()
    # fit the tokenizer on the documents
    tokenizer.fit_on_texts(train_docs)
    # encode training data set
    Xtrain = tokenizer.texts_to_matrix(train_docs, mode=mode)
    # encode training data set
    #Xvalid = tokenizer.texts_to_matrix(valid_docs, mode=mode)
    # encode testing data set
    Xtest = tokenizer.texts_to_matrix(test_docs, mode=mode)
    return Xtrain, Xtest

In [ ]:
modes = ['binary', 'count', 'tfidf', 'freq']
print(y_train)
results = DataFrame()
for mode in modes:
    Xtrain, Xtest = prepare_data(X_train['PreProcessedNews'], X_test['PreProcessedNews'], mode)
    
     # prepare data for mode
    #print(Xtrain)
    results[mode] = evaluate_mode(Xtrain, y_train, Xtest, y_test)
 # summarize results
    print(results.describe())
 # plot results
    results.boxplot()
    pyplot.show()

#X_train, X_valid, X_test = prepare_data(X_train, X_valid, X_test, mode)

[0 1 0 ... 1 0 1]
Epoch 1/10
38/38 - 2s - loss: 0.7026 - accuracy: 0.5004 - val_loss: 0.6871 - val_accuracy: 0.5617
Epoch 2/10
38/38 - 2s - loss: 0.3447 - accuracy: 0.9437 - val_loss: 0.7465 - val_accuracy: 0.5139
Epoch 3/10
38/38 - 2s - loss: 0.0962 - accuracy: 0.9840 - val_loss: 0.8232 - val_accuracy: 0.5466
Epoch 4/10
38/38 - 2s - loss: 0.0173 - accuracy: 1.0000 - val_loss: 0.8832 - val_accuracy: 0.5592
Epoch 5/10
38/38 - 2s - loss: 0.0076 - accuracy: 1.0000 - val_loss: 0.9264 - val_accuracy: 0.5416
Epoch 6/10
38/38 - 2s - loss: 0.0042 - accuracy: 1.0000 - val_loss: 0.9541 - val_accuracy: 0.5491
Epoch 7/10
38/38 - 2s - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.9863 - val_accuracy: 0.5491
Epoch 8/10
38/38 - 2s - loss: 0.0016 - accuracy: 1.0000 - val_loss: 1.0105 - val_accuracy: 0.5592
Epoch 9/10
38/38 - 2s - loss: 0.0011 - accuracy: 1.0000 - val_loss: 1.0359 - val_accuracy: 0.5642
Epoch 10/10
38/38 - 1s - loss: 7.4811e-04 - accuracy: 1.0000 - val_loss: 1.0550 - val_accuracy: 0.55

This repo contains an introduction to [Jupyter](https://jupyter.org) and [IPython](https://ipython.org).

Outline of some basics:

* [Notebook Basics](../examples/Notebook/Notebook%20Basics.ipynb)
* [IPython - beyond plain python](../examples/IPython%20Kernel/Beyond%20Plain%20Python.ipynb)
* [Markdown Cells](../examples/Notebook/Working%20With%20Markdown%20Cells.ipynb)
* [Rich Display System](../examples/IPython%20Kernel/Rich%20Output.ipynb)
* [Custom Display logic](../examples/IPython%20Kernel/Custom%20Display%20Logic.ipynb)
* [Running a Secure Public Notebook Server](../examples/Notebook/Running%20the%20Notebook%20Server.ipynb#Securing-the-notebook-server)
* [How Jupyter works](../examples/Notebook/Multiple%20Languages%2C%20Frontends.ipynb) to run code in different languages.

You can also get this tutorial and run it on your laptop:

    git clone https://github.com/ipython/ipython-in-depth

Install IPython and Jupyter:

with [conda](https://www.anaconda.com/download):

    conda install ipython jupyter

with pip:

    # first, always upgrade pip!
    pip install --upgrade pip
    pip install --upgrade ipython jupyter

Start the notebook in the tutorial directory:

    cd ipython-in-depth
    jupyter notebook